# Sample Siamese Network Code


**The training was interrupted, the network reaches on average accuracies larger than 0.75!**

Additionally a Random Forest and a Logistic Regression were fitted, but accuracies were notably below than the ones obtained in the Siamese Network.

The code were trained with positve and negative samples, postive was a seen item followed by a purchase in the same session, the negative the contrary.

In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random

In [3]:
#read all the data:
items = pd.read_csv("dressipi_recsys2022/item_features.csv")
candidate = pd.read_csv("dressipi_recsys2022/candidate_items.csv")
purchase =  pd.read_csv("dressipi_recsys2022/train_purchases.csv")
sessions =  pd.read_csv("dressipi_recsys2022/train_sessions.csv")
sessions_test = pd.read_csv("dressipi_recsys2022/test_final_sessions.csv")
leaderboard = pd.read_csv("dressipi_recsys2022/test_leaderboard_sessions.csv")

In [4]:
items.item_id.nunique()

23691

In [5]:
#set the date column to a date format
purchase['date'] = pd.to_datetime(purchase['date'])
#list of unique items:
items_unique = sessions.item_id.unique()

In [6]:
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]

#top 99 most sold items in the last month of the training data
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [7]:
sessions_item_dict = {}
item_sessions_dict = {}
for i,x in tqdm(enumerate(zip(sessions['session_id'], sessions['item_id']))):
    session, item = x
    if session not in sessions_item_dict:
        sessions_item_dict[session] = {}
    
    if item not in item_sessions_dict:
        item_sessions_dict[item] = {}
        
    if item not in sessions_item_dict[session]:
        sessions_item_dict[session][item] = 0
    
    if session not in item_sessions_dict[item]:
        item_sessions_dict[item][session] = 0
        
    sessions_item_dict[session][item] += 1
    item_sessions_dict[item][session] += 1

4743820it [00:15, 309445.76it/s]


In [6]:
sessions_item_dict_test = {}
item_sessions_dict_test = {}
for i,x in enumerate(zip(sessions_test['session_id'], sessions_test['item_id'])):
    session, item = x
    if session not in sessions_item_dict_test:
        sessions_item_dict_test[session] = {}
    
    if item not in item_sessions_dict_test:
        item_sessions_dict_test[item] = {}
        
    if item not in sessions_item_dict_test[session]:
        sessions_item_dict_test[session][item] = 0
    
    if session not in item_sessions_dict_test[item]:
        item_sessions_dict_test[item][session] = 0
        
    sessions_item_dict_test[session][item] += 1
    item_sessions_dict_test[item][session] += 1

In [7]:
sessions_item_dict_lad = {}
item_sessions_dict_lad = {}
for i,x in enumerate(zip(leaderboard['session_id'], leaderboard['item_id'])):
    session, item = x
    if session not in sessions_item_dict_lad:
        sessions_item_dict_lad[session] = {}
    
    if item not in item_sessions_dict_lad:
        item_sessions_dict_lad[item] = {}
        
    if item not in sessions_item_dict_lad[session]:
        sessions_item_dict_lad[session][item] = 0
    
    if session not in item_sessions_dict_lad[item]:
        item_sessions_dict_lad[item][session] = 0
        
    sessions_item_dict_lad[session][item] += 1
    item_sessions_dict_lad[item][session] += 1

In [8]:
def generate_positive(max_num = 10):
    '''
    Generates Samples of Products seen in the same Session
    with a maximum amount of max_num items per item
    '''
    positive = []
    positive_dict = {}
    for item in tqdm(item_sessions_dict):
        cont = 0
        for session in item_sessions_dict[item]:
            for elem in sessions_item_dict[session]:
                positive.append([item,elem])
                cont += 1
                if item not in positive_dict:
                    positive_dict[item] = []
                positive_dict[item].append(elem)
                if(cont == max_num): break
            if(cont == max_num): break
                
    return positive, positive_dict

positive_sample,positive_dict = generate_positive() #generates data of positive samples    

100%|██████████| 23496/23496 [00:00<00:00, 37896.71it/s]


In [9]:
def generate_negative(max_num = int(len(positive_sample)/len(items_unique))):
    '''Generates Samples of Products not seen in the same Session
    with a maximum amount of max_num items per item'''
    negative = []
    for item in tqdm(items_unique):
        cont = 0
        while(cont < max_num):
            negative_sample = random.choice(items_unique)
            if negative_sample not in positive_dict[item] and negative_sample != item:
                negative.append([item,negative_sample])
                cont += 1
    return negative   

negative_sample = generate_negative()

100%|██████████| 23496/23496 [00:01<00:00, 21151.69it/s]


In [10]:
def get_item_features():
    items_features = {}
    new_item = 2
    features = []
    for i in tqdm(range(len(items))):
        if(items.iloc[i].item_id == new_item):
            features.append(items.iloc[i].feature_category_id)
            features.append(items.iloc[i].feature_value_id)
        else:
            items_features[new_item] = features
            new_item = items.iloc[i].item_id
            features = []
            features.append(items.iloc[i].feature_category_id)
            features.append(items.iloc[i].feature_value_id)
    items_features[new_item] = features
    return items_features
item_features = get_item_features()

100%|██████████| 471751/471751 [02:10<00:00, 3608.31it/s]


The largest number of description is:

In [11]:
maxi = 0
for i in tqdm(items.item_id):
    if(len(item_features[i]) > maxi): maxi = len(item_features[i])
maxi

100%|██████████| 471751/471751 [00:00<00:00, 873894.69it/s] 


66

In [12]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [13]:
def generate_dataset():
    full_dataset = np.vstack((positive_sample,negative_sample))
    full_dataset = pd.DataFrame(full_dataset)
    full_dataset['label'] = 0
    full_dataset['label'].iloc[0:len(positive_sample)-1] = 1
    train = full_dataset.iloc[:,0:2]
    test = full_dataset.iloc[:,2:3]    
    X_train,X_val,y_train,y_val = train_test_split(train,test,test_size = 0.2,random_state = 42,shuffle = True)
    return X_train,X_val,y_train,y_val  

In [14]:
X_train,X_val,y_train,y_val = generate_dataset()

In [15]:
def generate_feature_dataset(df):
    max_length = 132
    feature_vector = []
    for i in tqdm(range(len(df))):
        x1 = df.iloc[i][0]
        x2 = df.iloc[i][1]
        x1_features = item_features[x1]
        x2_features = item_features[x2]
        combine_features = np.zeros(max_length)
        combine_features[0:len(x1_features)] = x1_features
        combine_features[int(max_length/2):int(max_length/2)+len(x2_features)] = x2_features
        feature_vector.append(combine_features)
    return feature_vector
    

In [16]:
X_train_feature = generate_feature_dataset(X_train)
X_test_feature = generate_feature_dataset(X_val)

100%|██████████| 86400/86400 [00:17<00:00, 4952.46it/s]


### Logistic Regression:

In [17]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train_feature,y_train)

C:\Users\marct\anaconda3\envs\TFM\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\marct\anaconda3\envs\TFM\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [18]:
print(f"The training accuracy is {model.score(X_train_feature, y_train)}")
print(f"The validation accuracy is {model.score(X_test_feature, y_val)}")

The training accuracy is 0.5480934496148705
The validation accuracy is 0.5457175925925926


Note that we do require a bit more of iterations to fully converge.

In [19]:
probabilities = model.predict_proba(X_test_feature)
good_matches = []
for i in range(len(probabilities)):
    if(probabilities[i,0]<= 0.40): good_matches.append(i)

In [20]:
len(good_matches)

4198

Alguna coseta es pot fer segur aquí veient això.

### Random Forests:

In [21]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train_feature,y_train)

C:\Users\marct\AppData\Local\Temp\ipykernel_11204\4279261279.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_feature,y_train)


RandomForestClassifier(random_state=0)

In [22]:
print(f"The training accuracy is {model.score(X_train_feature, y_train)}")
print(f"The validation accuracy is {model.score(X_test_feature, y_val)}")

The training accuracy is 0.9998032396020811
The validation accuracy is 0.7235532407407408


In [23]:
probabilities = model.predict_proba(X_test_feature)
good_matches = []
for i in range(len(probabilities)):
    if(probabilities[i,0]<= 0.35):
        good_matches.append(i)

In [24]:
matching = {}
first = np.array(X_train[0])
second = np.array(X_train[1])
for i in good_matches:
    if(first[i] not in matching):
        matching[first[i]] = []
    if(second[i] not in matching):
        matching[second[i]] = []
    matching[first[i]].append(second[i])
    matching[second[i]].append(first[i])

In [25]:
def generate_solution_rf():
    
    solution = [[]]
    
    for x in tqdm(leaderboard.session_id.unique()):
        seen = 0
        for k in sessions_item_dict_lad[x]:
            if k in matching:
                for j in matching[k]:
                    solution.append([int(x),int(j),seen+1])
                    seen += 1
                    if(seen == 100): break
                if(seen == 100): break
            if(seen == 100): break
        for j in range(100-seen):
            solution.append([int(x),int(top_seller[j]),seen+1])
            seen += 1
    return solution


In [26]:
solution = generate_solution_rf()

100%|██████████| 50000/50000 [00:09<00:00, 5363.74it/s] 


In [27]:
solution.pop(0)
solution = pd.DataFrame(solution, columns = ['session_id','item_id', 'rank']).reset_index(drop = True)

In [28]:
solution_csv = solution.to_csv
solution.to_csv(r'solution_csv.csv',index = False)
#first simple dummy solution:

# Siamese (ConvNet):

In [29]:
#####Let's Proceed now to generate the actual Siamesian Network############
#https://keras.io/examples/vision/siamese_contrastive/

In [30]:
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

In [31]:
###Now let's define the Siamese Network:
input_size = 66

In [32]:
input = layers.Input((input_size,1))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv1D(128, 2, strides = 2, activation="tanh")(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((input_size,1))
input_2 = layers.Input((input_size,1))

tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])

normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)

output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)

siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)


In [33]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.

      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [34]:
margin  = 1
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 66, 1)]      0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 66, 1)]      0           []                               
                                                                                                  
 model (Functional)             (None, 33, 16)       11252       ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1, 16)        0           ['model[0][0]',            

In [37]:
epochs = 10
batch_size = 256
margin = 1  # Margin for constrastive loss.

In [38]:
X_train_feature_np =  np.asarray(X_train_feature)
X_train_feature1 = list(X_train_feature_np[:,0:66])
X_train_feature2 = list(X_train_feature_np[:,66:])
X_test_feature_np = np.asarray(X_test_feature)
X_test_feature1 = list(X_test_feature_np[:,0:66])
X_test_feature2 = list(X_test_feature_np[:,66:])

In [39]:
history = siamese.fit(
    [np.array(X_train_feature1), np.array(X_train_feature2)],
    np.array(y_train.astype("float32")),
    validation_data=([np.array(X_test_feature1),np.array(X_test_feature2)], np.array(y_val.astype("float32"))),
    batch_size=batch_size,
    epochs=epochs,
)

Epoch 1/10
1350/1350 [==============================] - 35s 24ms/step - loss: 0.2254 - accuracy: 0.6097 - val_loss: 0.2544 - val_accuracy: 0.5133
Epoch 2/10
1350/1350 [==============================] - 31s 23ms/step - loss: 0.2231 - accuracy: 0.6175 - val_loss: 0.2352 - val_accuracy: 0.6001
Epoch 3/10
1350/1350 [==============================] - 31s 23ms/step - loss: 0.2199 - accuracy: 0.6277 - val_loss: 0.2203 - val_accuracy: 0.6264
Epoch 4/10
1350/1350 [==============================] - 42s 31ms/step - loss: 0.2184 - accuracy: 0.6320 - val_loss: 0.2338 - val_accuracy: 0.6197
Epoch 5/10
1350/1350 [==============================] - 40s 29ms/step - loss: 0.2171 - accuracy: 0.6357 - val_loss: 0.2178 - val_accuracy: 0.6347
Epoch 6/10
1350/1350 [==============================] - 41s 30ms/step - loss: 0.2163 - accuracy: 0.6366 - val_loss: 0.2164 - val_accuracy: 0.6341
Epoch 7/10
1350/1350 [==============================] - 41s 31ms/step - loss: 0.2170 - accuracy: 0.6346 - val_loss: 0.2201 -

In [ ]:
#history = siamese.fit(
  #  [X_train_features[0:67], X_train_features[67:],
  #  np.array(y_train.astype("float32")),
  #  validation_data=([np.array(X_val.astype("float32"))[:,0], np.array(X_val.astype("float32"))[:,1]], np.array(y_val.astype("float32"))),
  #  batch_size=batch_size,
  #  epochs=epochs,
#)

In [ ]:
def plt_metric(history, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(history=history.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history.history, metric="loss", title="Constrastive Loss")

# Testing:

In [ ]:
items_unique = sessions_test.item_id.unique()

In [ ]:
sessions_item_dict = {}
item_sessions_dict = {}
for i,x in enumerate(zip(sessions_test['session_id'], sessions_test['item_id'])):
    session, item = x
    if session not in sessions_item_dict:
        sessions_item_dict[session] = {}
    
    if item not in item_sessions_dict:
        item_sessions_dict[item] = {}
        
    if item not in sessions_item_dict[session]:
        sessions_item_dict[session][item] = 0
    
    if session not in item_sessions_dict[item]:
        item_sessions_dict[item][session] = 0
        
    sessions_item_dict[session][item] += 1
    item_sessions_dict[item][session] += 1

In [ ]:
def generate_positive(max_num = 25):
    '''
    Generates Samples of Products seen in the same Session
    with a maximum amount of max_num items per item
    '''
    positive = []
    positive_dict = {}
    for item in tqdm(item_sessions_dict):
        cont = 0
        for session in item_sessions_dict[item]:
            for elem in sessions_item_dict[session]:
                positive.append([item,elem])
                cont += 1
                if item not in positive_dict:
                    positive_dict[item] = []
                positive_dict[item].append(elem)
                if(cont == max_num): break
            if(cont == max_num): break
                
    return positive, positive_dict

positive_sample,positive_dict = generate_positive() #generates data of positive samples    

In [ ]:
def generate_negative(max_num = 25):
    '''Generates Samples of Products not seen in the same Session
    with a maximum amount of max_num items per item'''
    negative = []
    for item in tqdm(items_unique):
        for i in range(max_num):
            negative_sample = random.choice(items_unique)
            if negative_sample not in positive_dict[item] and negative_sample != item:
                negative.append([item,negative_sample])
    return negative   

negative_sample = generate_negative()

In [ ]:
def generate_dataset():
    full_dataset = np.vstack((positive_sample,negative_sample))
    full_dataset = pd.DataFrame(full_dataset)
    full_dataset['label'] = 0
    full_dataset['label'].iloc[0:len(positive_sample)-1] = 1
    train = full_dataset.iloc[:,0:2]
    test = full_dataset.iloc[:,2:3]    
    X_train,X_val,y_train,y_val = train_test_split(train,test,test_size = 0.99999,random_state = 42,shuffle = True)
    return X_train,X_val,y_train,y_val  
X_train,X_test,y_train,y_test = generate_dataset()

In [ ]:
results = siamese.evaluate([np.array(X_test.astype("float32"))[:,0], np.array(X_test.astype("float32"))[:,1]], np.array(y_test.astype("float32")))
print("test loss, test acc:", results)

In [ ]:
predictions = siamese.predict([np.array(X_test.astype("float32"))[:,0], np.array(X_test.astype("float32"))[:,1]])


In [ ]:
predictions